In [39]:
import numpy as np
import pandas as pd
import psycopg2
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from matplotlib import pyplot as plt
from meteocalc import heat_index, dew_point, wind_chill
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, ExtraTreesRegressor, \
    AdaBoostRegressor, StackingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [40]:
def convert_dates(df):
    if 'data' in df.columns:
        # Scomposizione in componenti separate
        df['year'] = df['data'].dt.year
        df['month'] = df['data'].dt.month
        df['day'] = df['data'].dt.day
        df['hour'] = df['data'].dt.hour
        # df['hour'] = df['data'].dt.hour
        df['giorno_settimana'] = df['data'].dt.dayofweek

        # Rimozione della colonna datetime originale
        df = df.drop('data', axis=1)
        sorted_data = df.sort_values(by=['year', 'month', 'day', 'fascia_oraria'])
        df = sorted_data
    return df

In [41]:
def calculate_additional_weather_features(df):
    # Conversione della temperatura da Celsius a Fahrenheit per meteocalc
    df['temp_f'] = df['temperature_2m'] * 9 / 5 + 32
    df['wind_speed_mph'] = df['wind_speed_10m'] * 2.237  # conversione da m/s a mph

    # Applicazione della funzione di calcolo solo dove ha senso fisicamente
    df['heat_index'] = df.apply(
        lambda row: heat_index(temperature=row['temp_f'], humidity=row['relative_humidity_2m'])
        if row['temp_f'] >= 80 and row['relative_humidity_2m'] >= 40 else row['temp_f'], axis=1)

    df['wind_chill'] = df.apply(
        lambda row: wind_chill(temperature=row['temp_f'], wind_speed=row['wind_speed_mph'])
        if row['temp_f'] <= 50 and row['wind_speed_mph'] >= 3 else row['temp_f'], axis=1)

    # Rimozione delle colonne temporanee
    df.drop(columns=['temp_f', 'wind_speed_mph'], inplace=True)

    return df

In [52]:
## Moduli necessari per il caricamento e la pulizia dei dati

import psycopg2
import pandas as pd

# Parametri di connessione al database
db_params = {
    'host': 'localhost',
    'database': 'HARPA',
    'user': 'user',
    'password': 'password',
    'port': '5432',
}

# Query SQL per la SELECT
select_query = 'SELECT * FROM harpa.aggregazione_ora WHERE 1=1 /** AND EXTRACT(YEAR from giorno) = 2023 **/ ORDER BY data ASC'

# Funzione per eseguire la query e restituire un DataFrame
def fetch_data_to_dataframe(query, connection_params):
    # Connessione al database
    connection = psycopg2.connect(**connection_params)
    
    # Esecuzione della query e caricamento dei dati nel DataFrame
    data_frame = pd.read_sql_query(query, connection)
    
    # Chiusura della connessione
    connection.close()
    
    return data_frame

# Eseguire la query e ottenere il DataFrame
df = fetch_data_to_dataframe(select_query, db_params)


# Visualizzare il DataFrame
# print(df)



C:\Users\franc\AppData\Local\Temp\ipykernel_14840\800271996.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_frame = pd.read_sql_query(query, connection)


In [43]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [53]:
df = df.drop(columns=['is_smartworking'])
df = df.dropna(axis=0, how='any')


In [54]:
pca = PCA()
pca.fit(df)

explained_variance_ratio = pca.explained_variance_ratio_


DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DTy

In [ ]:
pca = PCA()
pca.fit(df)

cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
